<a href="https://colab.research.google.com/github/DeepOzean/generative_summarization/blob/master/FineTune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import gzip
import math
import numpy as np
import pandas as pd

train_test_split = 0.95

In [2]:
!git clone -b gpt2 https://github.com/DeepOzean/generative_summarization.git gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 264 (delta 136), reused 249 (delta 129), pack-reused 0
Receiving objects: 100% (264/264), 4.39 MiB | 6.54 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [0]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
cd gpt-2

/content/gpt-2


In [5]:
!pip3 install -r requirements.txt
!python download_model.py 117M

     |████████████████████████████████| 604kB 17.8MB/s 
     |████████████████████████████████| 51kB 21.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
  Found existing installation: regex 2018.1.10
    Uninstalling regex-2018.1.10:
      Successfully uninstalled regex-2018.1.10
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
Fetching checkpoint: 1.00kit [00:00, 631kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 36.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 868kit/s]                                          

In [0]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz

--2019-05-23 12:50:05--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68275834 (65M) [application/x-gzip]
Saving to: ‘reviews_Sports_and_Outdoors_5.json.gz’

ports_and_Outdoors_  46%[========>           ]  29.96M  6.28MB/s    eta 6s     

In [0]:
df = getDF('reviews_Sports_and_Outdoors_5.json.gz')

In [0]:
df_reviewText = df['reviewText']
n = len(df_reviewText)
n_train = int(np.floor(n*train_test_split))
df_reviewText_train = df_reviewText[0:n_train]
df_reviewText_test = df_reviewText[n_train+1:]

In [0]:
df_summary = df['summary']
df_summary_train = df_summary[0:n_train]
df_summary_test = df_summary[n_train+1:]

In [0]:
df_summary.head(20)

In [0]:
corpus = ''
for i in range(len(df_summary_train)):
  corpus += df_reviewText_train[i]
  corpus += ' TL;DR: '
  corpus += df_summary_train[i]
  corpus += ' [End] '
  corpus += '\n'
corpus += '\n\n'

with open('review_and_summary.txt', 'w') as f:
  f.write(corpus)
  
!PYTHONPATH=src ./encode.py review_and_summary.txt review_and_summary.txt.npz

In [0]:
corpus_test = ''
for review in df_reviewText_test:
  corpus_test += review
  corpus_test += ' TL;DR: '
  corpus_test += '\n'
corpus_test += '\n\n'

with open('review_test.txt', 'w') as f:
  f.write(corpus_test)

In [0]:
corpus_test_conditioning = ''
count = 0
for i in range(n_train+1,n_train+len(df_reviewText_test)):
  corpus_test_conditioning += df_reviewText_test[i]
  corpus_test_conditioning += ' TL;DR: '
  corpus_test_conditioning += df_summary_test[i]
  corpus_test_conditioning += ' [End] '
  if i % 5 == 0:
    corpus_test_conditioning += df_reviewText_test[i]
    corpus_test_conditioning += ' TL;DR: '
    corpus_test_conditioning += '\n'
corpus_test_conditioning += '\n\n'

with open('review_test_conditioning.txt', 'w') as f:
  f.write(corpus_test_conditioning)

In [0]:
print(corpus_test_conditioning)

In [0]:
!PYTHONPATH=src ./train.py --dataset review_and_summary.txt.npz --sample_every=500 --learning_rate=0.0001 --stop_after=2500

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
#!python3 src/interactive_conditional_samples_read_file.py --file review_test.txt --top_k 4 --temperature 0.1 --length 100 --nsamples 3

In [0]:
!python3 src/interactive_conditional_samples_read_file.py --file review_test_conditioning.txt --top_k 40 --temperature 0.9 --length 1000 --nsamples 3